In [1]:
import random
import numpy as np
import pandas as pd
import math
# import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

import time
from IPython.utils.io import capture_output

In [2]:
# Rastrigin function
def rastrigin(individual):
    n = len(individual)
    A = 10
    return A * n + sum(x**2 - A * math.cos(2 * math.pi * x) for x in individual),

# Ackley function
def ackley(individual):
    n = len(individual)
    sum1 = sum(x**2 for x in individual)
    sum2 = sum(math.cos(2 * math.pi * x) for x in individual)
    return -20 * math.exp(-0.2 * math.sqrt(sum1 / n)) - math.exp(sum2 / n) + 20 + math.e,

# Rosenbrock function
def rosenbrock(individual):
    n = len(individual)
    return sum(100 * (individual[i+1] - individual[i]**2)**2 + (individual[i] - 1)**2 for i in range(n - 1)),

In [3]:
# Simulated annealing specific steps
def simulated_annealing(toolbox, ngen=100, temp=100.0, cooling_rate=0.95, pop_size=100):
    population = toolbox.population(n=pop_size)
    halloffame = tools.HallOfFame(1)
    logbook = tools.Logbook()
    logbook.header = ["gen", "evals", "fitness"]

    # Evaluate initial population
    for ind in population:
        ind.fitness.values = toolbox.evaluate(ind)
        
    for gen in range(ngen):
        temp *= cooling_rate
        for ind in population:
            neighbor = toolbox.clone(ind)
            toolbox.mutate(neighbor)
            del neighbor.fitness.values
            neighbor.fitness.values = toolbox.evaluate(neighbor)

            delta_e = neighbor.fitness.values[0] - ind.fitness.values[0]
            if delta_e < 0 or random.random() < np.exp(-delta_e / temp):
                ind[:] = neighbor
                ind.fitness.values = neighbor.fitness.values

        halloffame.update(population)
        record = {"gen": gen, "evals": len(population), "fitness": halloffame[0].fitness.values[0]}
        logbook.record(**record)
        print(logbook.stream)

    return halloffame, logbook

# Rastrigin

In [4]:
df_sol = pd.DataFrame()
temperature = 154
cooling = 1.85

lb = -5.12
up = 5.12

In [5]:
n_dim = 5
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim) 
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", rastrigin)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol['SA_rastrigin_5'] = pd.Series(fill)
    df_sol['SA_rastrigin_time_5'] = pd.Series(fill_t)

In [6]:
n_dim = 10
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation
        toolbox.register("evaluate", rastrigin)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol['SA_rastrigin_10'] = pd.Series(fill)
    df_sol['SA_rastrigin_time_10'] = pd.Series(fill_t)

In [7]:
n_dim = 20
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", rastrigin)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol['SA_rastrigin_20'] = pd.Series(fill)
    df_sol['SA_rastrigin_time_20'] = pd.Series(fill_t)

In [8]:
df_sol.mean().round(2)

SA_rastrigin_5           11.71
SA_rastrigin_time_5       2.22
SA_rastrigin_10          62.73
SA_rastrigin_time_10      2.64
SA_rastrigin_20         194.37
SA_rastrigin_time_20      3.56
dtype: float64

In [9]:
df_sol.min().round(2)

SA_rastrigin_5            8.11
SA_rastrigin_time_5       2.12
SA_rastrigin_10          54.81
SA_rastrigin_time_10      2.58
SA_rastrigin_20         178.32
SA_rastrigin_time_20      3.53
dtype: float64

# Ackley

In [10]:
df_sol_A = pd.DataFrame()
temperature = 95
cooling = 1.25

lb = -32.786
up = 32.786

In [11]:
n_dim = 5
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", ackley)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol_A['SA_ackley_5'] = pd.Series(fill)
    df_sol_A['SA_ackley_time_5'] = pd.Series(fill_t)

In [12]:
n_dim = 10
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", ackley)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol_A['SA_ackley_10'] = pd.Series(fill)
    df_sol_A['SA_ackley_time_10'] = pd.Series(fill_t)

In [13]:
n_dim = 20
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", ackley)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol_A['SA_ackley_20'] = pd.Series(fill)
    df_sol_A['SA_ackley_time_20'] = pd.Series(fill_t)

In [14]:
df_sol_A.mean().round(2)

SA_ackley_5           9.39
SA_ackley_time_5      2.22
SA_ackley_10         16.97
SA_ackley_time_10     2.70
SA_ackley_20         19.18
SA_ackley_time_20     3.74
dtype: float64

In [15]:
df_sol_A.min().round(2)

SA_ackley_5           6.74
SA_ackley_time_5      2.18
SA_ackley_10         16.10
SA_ackley_time_10     2.68
SA_ackley_20         18.55
SA_ackley_time_20     3.67
dtype: float64

# Rosenbrock

In [16]:
df_sol_R = pd.DataFrame()
temperature = 90
cooling = 0.7

lb = -5
up = 10

In [17]:
n_dim = 5
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", rosenbrock)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol_R['SA_rosenbrock_5'] = pd.Series(fill)
    df_sol_R['SA_rosenbrock_time_5'] = pd.Series(fill_t)

In [18]:
n_dim = 10
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", rosenbrock)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol_R['SA_rosenbrock_10'] = pd.Series(fill)
    df_sol_R['SA_rosenbrock_time_10'] = pd.Series(fill_t)

In [19]:
n_dim = 20
with capture_output() as captured:
    np.random.seed(42)
    fill = []
    fill_t = []
    for i in range(10):
        # Define the individual and fitness
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        # Define the attribute (each dimension in the individual)
        toolbox = base.Toolbox()
        toolbox.register("attr_float", random.uniform, lb, up)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_float, n=n_dim)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define the evaluation, mutation, and simulated annealing steps
        toolbox.register("evaluate", rosenbrock)
        toolbox.register("mutate", tools.mutPolynomialBounded, low=lb, up=up, eta=20.0, indpb=0.2)

        # Population = 100
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        pop_size = 100
        pop = toolbox.population(n=pop_size)

        # Evaluate the initial population
        for ind in pop:
            ind.fitness.values = toolbox.evaluate(ind)

        # Run simulated annealing
        start_time = time.time() #time     
        hof, log = simulated_annealing(toolbox, ngen=1000, temp=temperature, cooling_rate=cooling, pop_size=pop_size)
        end_time = time.time() #time
        elapsed_time = end_time - start_time

        print("Best individual:", hof[0])
        print("Best fitness:", hof[0].fitness.values[0])
        fill.append(hof[0].fitness.values[0])
        fill_t.append(elapsed_time)
    df_sol_R['SA_rosenbrock_20'] =  pd.Series(fill)
    df_sol_R['SA_rosenbrock_time_20'] = pd.Series(fill_t)

In [20]:
df_sol_R.mean().round(2)

SA_rosenbrock_5           0.13
SA_rosenbrock_time_5      2.39
SA_rosenbrock_10          3.54
SA_rosenbrock_time_10     3.16
SA_rosenbrock_20         59.02
SA_rosenbrock_time_20     4.36
dtype: float64

In [21]:
df_sol_R.min().round(2)

SA_rosenbrock_5           0.02
SA_rosenbrock_time_5      2.31
SA_rosenbrock_10          1.99
SA_rosenbrock_time_10     2.98
SA_rosenbrock_20         45.15
SA_rosenbrock_time_20     4.33
dtype: float64